## Importing modules

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

## Dataset Preparation

In [ ]:
# Paths to the dataset (update with actual paths)
train_dir = 'training dir path'
val_dir = 'validation dir path'
test_dir = 'test dir path'

# Image and training configuration
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
CLASSES = 7  # Number of emotion classes (FER 2013 has 7 classes)

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

## Dataset split

In [ ]:
train_data = train_datagen.flow_from_directory(
    train_dir, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical'
)
val_data = val_datagen.flow_from_directory(
    val_dir, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical'
)

## Loading and Compiling pretrained model

In [ ]:
# Load Pre-trained MobileNetV2
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze base model layers
base_model.trainable = False

# Build the Model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(CLASSES, activation='softmax')
])

# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Training Model

In [ ]:
# Train the Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    steps_per_epoch=len(train_data),
    validation_steps=len(val_data),
)

# Unfreeze and Fine-tune the Base Model
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
fine_tune_history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,  # Fine-tuning for additional epochs
    steps_per_epoch=len(train_data),
    validation_steps=len(val_data),
)

## Evaluating Model

In [ ]:
# Evaluate on Test Data
test_data = val_datagen.flow_from_directory(
    test_dir, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='categorical'
)
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.2f}")

## Saving model

In [ ]:
# Save the Model
model.save('emotion_model_mobilenetv2.h5')